<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infection Rates per day

[this code available on Github](https://github.com/InTEGr8or/jupyter-fun/blob/master/nCov19.ipynb)

The percent rates in the sheet at the bottom are approximations. Data seems to be released about twice a day. Sometimes there is a lag, and they release it at different times.

Optimally, the time of the update would be taken into account and prorated per hour and multiplied by the number of hours difference but I am just starting to learn Python so there are probably a lot of improvements that could be made.

Data is collected by Johns Hopkins in Baltimore and published here: [nCov19 contagion](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)

Now a [Time Series Table](https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/htmlview?usp=sharing&sle=true) is available, and a [Feature Layers](https://gisanddata.maps.arcgis.com/home/item.html?id=c0b356e20b30490c8b8b4c7bb9554e7c) appears to be available but it requires authentication.

The single-sheet Time Series is a much cleaner data source and _doesn't require repeated reauthentication_ so I'm reworking it to use that and we don't have a percent change right now, until I figure out how to use Pandas properly.

## This first section sets up the imports and some parsing functions.


In [0]:
try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import requests
import numpy
from dateutil import parser
from datetime import datetime
import pandas as pd

tsc_csv = "https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/export?format=csv"
tsc_html = "https://docs.google.com/spreadsheets/u/1/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/htmlview?usp=sharing&sle=true#"

pd.set_option('display.max_rows', 200)

def is_date(text):
  try:
    s = parser.parse(text)
    return True
  except:
    return False

def hotten(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    if '%' in s:
      print(s)
      is_max = float(s.replace('%', '')) > 20
      return ['background-color: yellow' if v else '' for v in is_max]
    else:
      return 'background-color: black'

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if ('%' in val and val.replace('%', '').isnumeric()) and (int(val.replace('%', '')) < 0) else 'black'
    return 'color: %s' % color

## Get Data, Loop Through and Print

This section is massively simplified with now that Johns Hopkins tidied up their data sources (Thanks Johns Hopkins tech people! _Way_ better)

I'm sure they have a lot of task on their hands and it's nice to see they can retrace over their first crack at the organization of the data that was coming in.

In [47]:
dfs = pd.read_csv(tsc_csv, dtype={0:'str', 1:'str', 2:'str'})
dfs.fillna('')
df = pd.DataFrame()
# First confirmed date in country (Est.)
df['Country'] = dfs['Country/Region'].replace('Mainland China', 'China').replace('United Arab Emirates', 'UAE')
df['Province/State'] = dfs['Province/State'].fillna('')
df['First Case'] = dfs['First confirmed date in country (Est.)']
df['Days Ago'] = datetime.now() - df['First Case'].apply(lambda date: parser.parse(date, fuzzy=True))
df['Days Ago'] = df['Days Ago'].apply(lambda days: str(days).split(' ')[0])
# print([col for col, idx in enumerate(dfs.columns) if is_date(col) ])

# for i, col in enumerate(dfs.columns):
#   if is_date(col):
#     print(col)
# for i, c in enumerate(dfs.columns):


# df = df.drop(labels=['Lat', 'Long'],axis=1)
# print(df.groupby([df.columns.where(c is_date(c)).dt.date]).mean())
# page = requests.get(tsc_html)

# df = df.reindex(sorted(df.columns, reverse=True), axis='columns')
# for idx, column in enumerate(df.columns):
#   if idx < len(df.columns) - 1 and is_date(column) and is_date(df.columns[idx + 1]):
#     next_col = df.columns[idx + 1]
#     # print(next_col)
#     if is_date(next_col) and parser.parse(column).day != parser.parse(next_col).day:
#       col_d = parser.parse(column)
#       col = col_d.strftime('%B')[:3] + col_d.strftime('%d')
#       df[col + '%'] = round((df[column] / df[next_col] * 100) - 100, 0)
#       df.rename(columns={column: col + "_" + col_d.strftime("%H:%m")}, inplace=True)
#       df.columns.insert(idx + 1, df[col + '%'])

# bs = BeautifulSoup(page.text)
# tpath = "//div[@id='sheets-viewport']/div //table"
# tables = bs.select('div#sheets-viewport div table')
# trc = tables[0].select("tr")
# th = trc[0].select('th')
# print(len(tables))
# print(th)
# print(trc)
# print(len(trc))

#.style.applymap(color_negative_red)
df#.sort_values(by=[df.columns[5],'Country/Region','Province/State'], ascending=[False, True, True])

[]


,Country,Province/State,First Case,Days Ago
0,China,Anhui,1/3/2020,34
1,China,Beijing,1/3/2020,34
2,China,Chongqing,1/3/2020,34
3,China,Fujian,1/3/2020,34
4,China,Gansu,1/3/2020,34
5,China,Guangdong,1/3/2020,34
6,China,Guangxi,1/3/2020,34
7,China,Guizhou,1/3/2020,34
8,China,Hainan,1/3/2020,34
9,China,Hebei,1/3/2020,34
